L'inconvénient avec cette approche est que si le lieu spécifié n'est pas reconnu, alors, on n'arrive pas à extraire des informations.

In [1]:
import requests
import geopandas as gpd
import folium
from shapely.geometry import Point

# Définir l'URL de l'API Overpass pour récupérer les données géospatiales
overpass_url = "https://overpass-api.de/api/interpreter"

# Définir la requête Overpass pour Youssoufia avec différents types de points d'intérêt
overpass_query = """
    [out:json];
    area[name="Youssoufia"]->.searchArea;
    (
        node["amenity"="pharmacy"](area.searchArea);
        node["name"="La Poste"](area.searchArea);
        node["name"="YouCode"](area.searchArea);
    );
    out center;
"""

# Envoyer la requête à l'API Overpass
response = requests.get(overpass_url, params={'data': overpass_query})

# Vérifier si la requête a réussi (statut 200 indique succès)
if response.status_code == 200:
    # Convertir la réponse JSON en un dictionnaire Python
    data = response.json()
    
    # Créer des listes de dictionnaires contenant les informations sur les différentes catégories
    pharmacies_data = []
    la_poste_data = []
    youcode_data = []
    
    for element in data['elements']:
        if element.get('tags', {}).get('amenity') == 'pharmacy':
            pharmacies_data.append({
                'name': element.get('tags', {}).get('name'),
                'latitude': element.get('lat'),
                'longitude': element.get('lon')
            })
        
        elif element.get('tags', {}).get('name') == 'La Poste':
            la_poste_data.append({
                'name': element.get('tags', {}).get('name'),
                'latitude': element.get('lat'),
                'longitude': element.get('lon')
            })
        elif element.get('tags', {}).get('name') == 'YouCode':
            youcode_data.append({
                'name': element.get('tags', {}).get('name'),
                'latitude': element.get('lat'),
                'longitude': element.get('lon')
            })
    
    # Créer des GeoDataFrames à partir des données pour chaque catégorie
    pharmacies_gdf = gpd.GeoDataFrame(pharmacies_data, geometry=gpd.points_from_xy([p['longitude'] for p in pharmacies_data], [p['latitude'] for p in pharmacies_data]))
    la_poste_gdf = gpd.GeoDataFrame(la_poste_data, geometry=gpd.points_from_xy([lp['longitude'] for lp in la_poste_data], [lp['latitude'] for lp in la_poste_data]))
    youcode_gdf = gpd.GeoDataFrame(youcode_data, geometry=gpd.points_from_xy([y['longitude'] for y in youcode_data], [y['latitude'] for y in youcode_data]))
    
    # Créer une carte Folium centrée sur les pharmacies (vous pouvez choisir une autre catégorie si vous le souhaitez)
    m = folium.Map(location=[pharmacies_gdf['latitude'].mean(), pharmacies_gdf['longitude'].mean()], zoom_start=14)
    
    # Ajouter les marqueurs des pharmacies sur la carte
    for idx, row in pharmacies_gdf.iterrows():
        folium.Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(m)
    
    # Ajouter les marqueurs de La Poste sur la carte
    for idx, row in la_poste_gdf.iterrows():
        folium.Marker([row['latitude'], row['longitude']], popup=row['name'], icon=folium.Icon(color='yellow')).add_to(m)
    
    # Ajouter les marqueurs de YouCode sur la carte
    for idx, row in youcode_gdf.iterrows():
        folium.Marker([row['latitude'], row['longitude']], popup=row['name'], icon=folium.Icon(color='purple')).add_to(m)
    
    # Afficher la carte
    display(m)
else:
    print("La requête a échoué. Statut :", response.status_code)


# Calcul de la distance entre deux points

In [2]:
!pip install geopy


In [3]:
from geopy.distance import geodesic


# Ajouter la fonction de calcul de distance
def calculate_distance(point1, point2):
    coord1 = (point1.y, point1.x)  # Convertir en tuple (latitude, longitude)
    coord2 = (point2.y, point2.x)
    return geodesic(coord1, coord2).meters  # Retourne la distance en mètres
# Récupérer les points d'intérêt sélectionnés par l'utilisateur (pharmacies, La Poste, YouCode)
selected_points = pharmacies_gdf['geometry'].tolist() + la_poste_gdf['geometry'].tolist() + youcode_gdf['geometry'].tolist()

# L'utilisateur doit sélectionner deux points d'intérêt par leur indice sur la carte
selected_indices = [0, 1]  # Exemple : l'utilisateur sélectionne les deux premiers points (pharmacies) pour commencer

# Obtenir les coordonnées des deux points sélectionnés
point1 = selected_points[selected_indices[0]]
point2 = selected_points[selected_indices[1]]

# Calculer la distance entre les deux points sélectionnés
distance = calculate_distance(point1, point2)

# Afficher la distance calculée
print(f"Distance entre {pharmacies_gdf.iloc[selected_indices[0]]['name']} et {pharmacies_gdf.iloc[selected_indices[1]]['name']}: {distance:.2f} mètres")

# Afficher la carte avec les marqueurs et la distance calculée
m = folium.Map(location=[pharmacies_gdf['latitude'].mean(), pharmacies_gdf['longitude'].mean()], zoom_start=14)

# Ajouter les marqueurs des pharmacies sur la carte
for idx, row in pharmacies_gdf.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(m)

# Ajouter les marqueurs de La Poste sur la carte
for idx, row in la_poste_gdf.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['name'], icon=folium.Icon(color='yellow')).add_to(m)

# Ajouter les marqueurs de YouCode sur la carte
for idx, row in youcode_gdf.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['name'], icon=folium.Icon(color='purple')).add_to(m)

# Tracer la ligne entre les deux points sélectionnés
folium.PolyLine(locations=[(point1.y, point1.x), (point2.y, point2.x)], color='blue').add_to(m)

# Afficher la carte
display(m)


Distance entre Pharmacie Dakhla صيدلية الداخلة et Pharmacie Al Amal صيدلية الأمل: 440.51 mètres
